In [1]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd

## Load DATA


### Observations


In [2]:
url = "https://raw.githubusercontent.com/Ash12H/seapopym-data/dev/data/hot/3_post_processed/hot_product.zarr"
hot_obs = xr.open_dataset(fsspec.get_mapper(url), engine="zarr")
hot_obs = hot_obs.rename({"lat": "latitude", "lon": "longitude"})
hot_obs

<xarray.Dataset> Size: 2MB
Dimensions:        (time: 659, latitude: 1, longitude: 1, depth: 8, frac: 5)
Coordinates:
  * depth          (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac           (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
  * latitude       (latitude) float64 8B 22.75
  * longitude      (longitude) int64 8B -158
  * time           (time) datetime64[ns] 5kB 1988-10-31 ... 2022-09-02
Data variables: (12/24)
    bsal           (time, latitude, longitude, depth) float64 42kB ...
    carb_migrant   (frac, time, latitude, longitude, depth) float64 211kB ...
    carb_resident  (frac, time, latitude, longitude, depth) float64 211kB ...
    chl            (time, latitude, longitude, depth) float64 42kB ...
    d12            (time, latitude, longitude, depth) float64 42kB ...
    dwt_migrant    (frac, time, latitude, longitude, depth) float64 211kB ...
    ...             ...
    sigma          (time, latitude, longitude, depth) float64 42kB ...
    temp           (time, latitude, longitude, depth) float64 42kB ...
    theta          (time, latitude, longitude, depth) float64 42kB ...
    wwt_migrant    (frac, time, latitude, longitude, depth) float64 211kB ...
    wwt_resident   (frac, time, latitude, longitude, depth) float64 211kB ...
    xmiss          (time, latitude, longitude, depth) float64 42kB ...

In [3]:
LATITUDE = hot_obs.latitude.data[0]
LONGITUDE = hot_obs.longitude.data[0]
START_TIME = pd.Timestamp(hot_obs.time.data[0]).strftime("%Y-%m-%d")
END_TIME = pd.Timestamp(hot_obs.time.data[-1]).strftime("%Y-%m-%d")
DELTA = 0.5

print(f"Latitude: {LATITUDE}\nLongitude: {LONGITUDE}\nStart Time: {START_TIME}\nEnd Time: {END_TIME}\nDelta: {DELTA}")

Latitude: 22.75
Longitude: -158
Start Time: 1988-10-31
End Time: 2022-09-02
Delta: 0.5


### CMEMS


In [4]:
FROM_API = False

In [5]:
# copernicusmarine.login()

coordinates = {
    "minimum_latitude": LATITUDE - DELTA,
    "maximum_latitude": LATITUDE + DELTA,
    "minimum_longitude": LONGITUDE - DELTA,
    "maximum_longitude": LONGITUDE + DELTA,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}


if FROM_API:
    bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
    bio.load()
    physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)
    physic.load()
else:
    bio = xr.load_dataset("../../data/phd/HOT/CMEMS/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1718150094521.nc")
    physic = xr.load_dataset("../../data/phd/HOT/CMEMS/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1718149967802.nc")

hot_data = xr.merge([bio, physic])
hot_data = hot_data.sel(time=slice(START_TIME, END_TIME)).sel(
    latitude=[LATITUDE], longitude=[LONGITUDE], method="nearest"
)
lat_attrs, lon_attrs = hot_data.latitude.attrs, hot_data.longitude.attrs
hot_data = hot_data.assign_coords({"latitude": [LATITUDE], "longitude": [LONGITUDE]})
hot_data.latitude.attrs, hot_data.longitude.attrs = lat_attrs, lon_attrs
hot_data

<xarray.Dataset> Size: 829kB
Dimensions:              (time: 9011, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * time                 (time) datetime64[ns] 72kB 1998-01-01 ... 2022-09-02
  * depth                (depth) float32 12B 1.0 2.0 3.0
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
Data variables: (12/13)
    mnkc_epi             (time, latitude, longitude) float32 36kB 0.2627 ... ...
    mnkc_hmlmeso         (time, latitude, longitude) float32 36kB 0.7987 ... ...
    mnkc_lmeso           (time, latitude, longitude) float32 36kB 2.157 ... 1...
    mnkc_mlmeso          (time, latitude, longitude) float32 36kB 1.095 ... 0...
    mnkc_mumeso          (time, latitude, longitude) float32 36kB 0.4632 ... ...
    mnkc_umeso           (time, latitude, longitude) float32 36kB 0.4843 ... ...
    ...                   ...
    zeu                  (time, latitude, longitude) float32 36kB 78.17 ... 8...
    zooc                 (time, latitude, longitude) float32 36kB 0.2832 ... ...
    T                    (time, depth, latitude, longitude) float32 108kB 23....
    U                    (time, depth, latitude, longitude) float32 108kB 0.0...
    V                    (time, depth, latitude, longitude) float32 108kB -0....
    pelagic_layer_depth  (time, depth, latitude, longitude) float32 108kB 117...
Attributes:
    Conventions:       CF-1.11
    title:             Global ocean low and mid trophic levels biomass conten...
    institution:       CLS
    source:            SEAPODYM-LMTL 3.0.0
    history:           Created on 2022-09-28
    references:        http://www.cls.fr; http://www.seapodym.eu
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-11T23:54:54.527Z

## Gathering Data & Save


In [6]:
hot_data.to_zarr("hot_cmems.zarr", mode="w")
hot_obs.to_zarr("hot_obs.zarr", mode="w")